# Entrena el carrito de montaña

[OpenAI Gym](http://gym.openai.com) ha sido diseñado de tal forma que todos los ambientes proveen la misma API - esto es, los mismos métodos `reset`, `step` y `render`, y las mismas abstracciones de **action space** y **observation space**. Así sería posible adaptar los mismos algoritmos de aprendizaje reforzado a diferentes ambientes con mínimos cambios al código.

## Instrucciones

Adapta nuestro algoritmo de aprendizaje reforzado para resolver el problema del carrito de montaña. Comienza con el código existente en [notebook.ipynb](../notebook.ipynb), substituye el nuevo ambiente, cambia las funciones de discretización de estado, e intenta hacer que el algoritmo existente entrene con mínimas modificaciones al código. Optimiza el resultado al ajustar los hiperparámetros.

1. Instalar OpenAI Gym:
Primero, asegúrate de tener gym instalado.

2. Importa las librerías necesarias y define el entorno de MountainCar-v0 que ya está disponible en Gym.

In [2]:
import sys
!{sys.executable} -m pip install gym 

import gym
import numpy as np
import random
from collections import defaultdict

3. Definición del entorno: Utiliza MountainCar-v0 de OpenAI Gym.

4. Discretización del espacio de observación: Dado que el espacio de observación es continuo, lo discretizamos utilizando bins.

In [3]:
# Crear el entorno de MountainCar-v0
env = gym.make('MountainCar-v0')

# Definir el rango de discretización para las observaciones
position_bins = np.linspace(-1.2, 0.6, 20)
velocity_bins = np.linspace(-0.07, 0.07, 20)

def discretize(obs):
    position, velocity = obs
    position_idx = np.digitize(position, position_bins)
    velocity_idx = np.digitize(velocity, velocity_bins)
    return position_idx, velocity_idx

5. Definición del algoritmo de Q-learning: Se define el algoritmo de Q-learning con una Q-Tabla inicializada.

In [4]:
# Definir los parámetros de Q-learning
alpha = 0.1  # Tasa de aprendizaje
gamma = 0.99  # Factor de descuento
epsilon = 0.1  # Tasa de exploración
episodes = 10000  # Número de episodios de entrenamiento


In [5]:
# Inicializar la Q-Tabla
Q = defaultdict(float)

# Definir la función para elegir una acción
def choose_action(state, epsilon):
    if random.random() < epsilon:
        return env.action_space.sample()
    else:
        q_values = [Q[(state, a)] for a in range(env.action_space.n)]
        return np.argmax(q_values)

6. Entrenamiento: Entrena el agente durante un número específico de episodios.

In [6]:
# Entrenar el agente con Q-learning
# Entrenar el agente con Q-learning
for episode in range(episodes):
    obs, _ = env.reset()
    state = discretize(obs)
    done = False
    total_reward = 0

    while not done:
        action = choose_action(state, epsilon)
        next_obs, reward, done, truncated, info = env.step(action)
        next_state = discretize(next_obs)

        if next_obs[0] >= 0.5:
            reward = 0
        else:
            reward = -1

        best_next_action = np.argmax([Q[(next_state, a)] for a in range(env.action_space.n)])
        td_target = reward + gamma * Q[(next_state, best_next_action)]
        Q[(state, action)] += alpha * (td_target - Q[(state, action)])

        state = next_state
        total_reward += reward

    if episode % 1000 == 0:
        print(f"Episode: {episode}, Total Reward: {total_reward}")

env.close()

c:\Users\HP\anaconda3\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode: 0, Total Reward: -10857
Episode: 1000, Total Reward: -508
Episode: 2000, Total Reward: -153
Episode: 3000, Total Reward: -163
Episode: 4000, Total Reward: -213
Episode: 5000, Total Reward: -195
Episode: 6000, Total Reward: -184
Episode: 7000, Total Reward: -189
Episode: 8000, Total Reward: -210
Episode: 9000, Total Reward: -223


7. Evaluación: Evalúa la política aprendida durante unos pocos episodios, renderizando el entorno para observar el comportamiento del agente.

In [7]:
# Evaluar la política aprendida
for episode in range(5):
    obs, _ = env.reset()
    state = discretize(obs)
    done = False
    total_reward = 0

    while not done:
        env.render()
        action = choose_action(state, 0)  # Exploit only
        next_obs, reward, done, truncated, info = env.step(action)
        state = discretize(next_obs)
        total_reward += reward

    print(f"Episode: {episode}, Total Reward: {total_reward}")

env.close()

Episode: 0, Total Reward: -96.0
Episode: 1, Total Reward: -138.0
Episode: 2, Total Reward: -143.0
Episode: 3, Total Reward: -125.0
Episode: 4, Total Reward: -230.0


c:\Users\HP\anaconda3\Lib\site-packages\gym\envs\classic_control\mountain_car.py:171: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("MountainCar-v0", render_mode="rgb_array")
  gym.logger.warn(


### Conclusión
En este ejercicio, se implemento y entreno un agente utilizando el algoritmo de Q-learning en el entorno MountainCar-v0 de **OpenAI** Gym. El objetivo del agente es aprender a controlar un carrito atrapado en un valle para alcanzar una bandera en la cima de la montaña.

* Este ejercicio demostró la eficacia del algoritmo de Q-learning en entornos con espacios de estado discretos. La discretización del espacio de observación permitió manejar la naturaleza continua del entorno MountainCar-v0.
* La política epsilon-greedy equilibró adecuadamente la exploración de nuevas acciones con la explotación del conocimiento adquirido, lo cual fue crucial para el aprendizaje efectivo del agente.
* Una correcta discretización del espacio de observación fue fundamental para el éxito del agente. Una discretización muy gruesa podría haber limitado la capacidad del agente para aprender diferencias sutiles en el entorno, mientras que una demasiado fina podría haber conducido a una explosión en el tamaño de la Q-Tabla.
* El ejercicio también destacó la necesidad de adaptar el código a las actualizaciones de la API de OpenAI Gym, particularmente en cómo se manejan los retornos de la función env.step().

En resumen, este ejercicio proporcionó una comprensión práctica de cómo implementar y ajustar el Q-learning en un entorno de simulación, destacando tanto los desafíos como las soluciones efectivas en el proceso de aprendizaje por refuerzo.